In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load data

In [81]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sample_sub = pd.read_csv('./data/gender_submission.csv')

In [82]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [83]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [84]:
sample_sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [85]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [86]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Will be approaching this problem with tree-based models in mind.

# Feature engineering

In [87]:
from sklearn import preprocessing

In [140]:
def transform_features(df):
    columns_to_drop = []
    
    sex_le = preprocessing.LabelEncoder()
    sex_le.fit(df['Sex'])
    
    df['SexEncoded'] = sex_le.transform(df['Sex'])
    
    columns_to_drop.extend(['Sex', 'Ticket', 'Cabin'])
    
    embarked_le = preprocessing.LabelEncoder()
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].value_counts().index[0])
    embarked_le.fit(df['Embarked'])
    
    df['EmbarkedEncoded'] = embarked_le.transform(df['Embarked'])
    
    columns_to_drop.append('Embarked')
    
    person_categories = [range(0, 5), range(5, 15), range(15, 25), range(25, 40), range(40, 60), range(60, 100)]
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    
    def encode_groups(row):
        return next(i for i,v in enumerate(person_categories) if int(row['Age']) in v) * 2 + row['SexEncoded'] 
    
    def 

    df['AgeGroup'] = df.apply(encode_groups, axis=1)
    
    columns_to_drop.extend(['SexEncoded', 'Name', 'PassengerId'])
    
    return df.drop(columns_to_drop, axis=1)

In [106]:
train_engineered = transform_features(train)

In [107]:
train_engineered.head(15)

,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedEncoded,AgeGroup
0,0,3,22.000000,1,0,7.2500,2,5
1,1,1,38.000000,1,0,71.2833,0,6
2,1,3,26.000000,0,0,7.9250,2,6
3,1,1,35.000000,1,0,53.1000,2,6
4,0,3,35.000000,0,0,8.0500,2,7
5,0,3,29.699118,0,0,8.4583,1,7
6,0,1,54.000000,0,0,51.8625,2,9
7,0,3,2.000000,3,1,21.0750,2,1
8,1,3,27.000000,0,2,11.1333,2,6
9,1,2,14.000000,1,0,30.0708,0,2


In [116]:
X = train_engineered.drop(['Survived'], axis=1)
y = train_engineered['Survived']

In [117]:
X.head(10)

,Pclass,Age,SibSp,Parch,Fare,EmbarkedEncoded,AgeGroup
0,3,22.000000,1,0,7.2500,2,5
1,1,38.000000,1,0,71.2833,0,6
2,3,26.000000,0,0,7.9250,2,6
3,1,35.000000,1,0,53.1000,2,6
4,3,35.000000,0,0,8.0500,2,7
5,3,29.699118,0,0,8.4583,1,7
6,1,54.000000,0,0,51.8625,2,9
7,3,2.000000,3,1,21.0750,2,1
8,3,27.000000,0,2,11.1333,2,6
9,2,14.000000,1,0,30.0708,0,2


In [118]:
y.head(10)

0    0
1    1
2    1
3    1
4    0
5    0
6    0
7    0
8    1
9    1
Name: Survived, dtype: int64

# Split training

In [119]:
from sklearn.model_selection import train_test_split

In [120]:
seed = 7
test_size = 0.33
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [128]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [129]:
model = XGBClassifier()
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [130]:
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 78.98%


/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [133]:
real_model = XGBClassifier()
real_model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [141]:
engineered_test = transform_features(test)

In [159]:
y_pred = model.predict(engineered_test)
predictions = [round(value) for value in y_pred]

/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [160]:
y_pred.size

418

In [162]:
sample_sub['Survived'] = y_pred

In [165]:
sample_sub.to_csv('sub.csv', index=False)